In [ ]:
import requests 
import pandas as pd 
import json 
import numpy as np
import pprint

from bs4 import BeautifulSoup

In [ ]:
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}

In [ ]:
api_key = 'ac80e2c89884220d09d71b0042b91aebc5f4cd1f'

In [ ]:
url = 'https://www.gamespot.com/api/reviews/'

In [ ]:
url_params = {
    'api_key': api_key,
    'format': 'json',
    'limit': 100,
            }


# dict_keys(['publish_date', 'update_date', 'review_type', 'id', 'authors', 'title', 'image', 'score', 'deck', 'good', 'bad', 'body', 'lede', 'game', 'releases', 'site_detail_url'])

In [ ]:
# running a functionto return all the request results as a json for each different url param

def data_call(url_params):
    response = requests.get(url,headers = header, params = url_params)
    res_json = json.loads(response.text)
    
    return res_json['results']


In [ ]:
columns = ['Title', 'Platform', 'Rating']
tester = pd.DataFrame(columns=columns)
tester.to_csv('gamespotdata.csv')

In [ ]:
# function so that when we loop through the different endpoints in the api it will store them in a dictionary
# then append an empty list of results with each dictionary entry so that it can easily be converted to a df

results = []

def parse_data(lst):
    global results
    for data in lst:
        if 'releases' in data.keys():
            for release in data['releases']:
                if 'Xbox One' in release['platform']:
                    obj = {'title':data['title'], 'platforms': release['platform'], 'score': data['score']}
                    results.append(obj)
    
    return results       




In [ ]:
cur = 0

#set up a while loop to go through and grab the result 

# will be going 100 at a time as that is what is allowed by the api for each page, the 2 functions take all the
# data from the page then put it into a list of dictionaries to convert to a df

while cur < 15049:
    url_params['offset'] = cur
    info = data_call(url_params)
    parse_data(info)
    
    cur += 100

In [ ]:
df = pd.DataFrame(results)

In [ ]:
titles = []
for title in df['title']:
    titles.append(title.replace(' Review',''))
        

In [ ]:
df.drop(columns='title', inplace=True)

In [ ]:
df.drop(columns='platforms',inplace=True)

In [ ]:
df['Title'].unique()

In [ ]:
df = df.drop_duplicates('Title')

In [ ]:
df.to_csv('gamespot_reviews.csv')